In [ ]:
# clinvar format
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
# 1	66926	3385321	AG	A	.	.	ALLELEID=3544463;CLNDISDB=Human_Phenotype_Ontology:HP:0000547,MONDO:MONDO:0019200,MeSH:D012174,MedGen:C0035334,OMIM:268000,OMIM:PS268000,Orphanet:791;CLNDN=Retinitis_pigmentosa;CLNHGVS=NC_000001.10:g.66927del;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNSIGSCV=SCV005419006;CLNVC=Deletion;CLNVCSO=SO:0000159;GENEINFO=OR4F5:79501;MC=SO:0001627|intron_variant;ORIGIN=0

In [ ]:
# 23andme format
# rsid,chromosome,position,genotype
# rs12564807,1,734462,AA

In [ ]:
# lets filter clinvar and keep only rows where the position is inside work/merged_rsids_positions.csv

In [ ]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Set, Tuple

# --- helpers ---

def load_rsids(path: str | Path) -> Set[str]:
    rs: Set[str] = set()
    with Path(path).open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            s = line.strip().lower()
            if s and s.startswith("rs"):
                # keep strictly rs<digits>
                m = re.match(r"^rs\d+$", s)
                if m:
                    rs.add(s)
    return rs

def load_positions_from_nonrsid_rows(path: str | Path) -> Set[Tuple[str, int]]:
    """
    File contains original 23andMe data rows (non-rsid), whitespace-separated:
      <id> <chromosome> <position> [genotype?]
    Return a deduped set of (chrom, pos_int).
    """
    pos: Set[Tuple[str, int]] = set()
    with Path(path).open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            s = line.strip()
            if not s or s.startswith("#"):
                continue
            parts = re.split(r"\s+", s)
            if len(parts) < 3:
                continue
            chrom, pos_str = parts[1], parts[2]
            try:
                pos.add((chrom, int(pos_str)))
            except ValueError:
                # skip weird rows
                continue
    return pos

def clinvar_info_rsids(info: str) -> Set[str]:
    """
    Extract rsIDs from ClinVar VCF INFO field (key 'RS=123,456', etc.).
    Returns {'rs123', 'rs456', ...} (lowercase).
    """
    out: Set[str] = set()
    if not info:
        return out
    for item in info.split(";"):
        if not item:
            continue
        if item.startswith("RS="):
            val = item[3:]
            for num in val.split(","):
                num = num.strip()
                if num.isdigit():
                    out.add(f"rs{num}".lower())
    return out

# --- main ---

def filter_clinvar_with_rsids_and_positions(
    vcf_in: str | Path = "downloads/clinvar.vcf",
    rsids_path: str | Path = "work/23andme_rsids.txt",
    nonrsid_grch37_path: str | Path = "work/23andme_nonrsid_grch37.txt",
    vcf_out: str | Path = "work/clinvar.23andme.vcf",
) -> dict:
    """
    Keep a ClinVar record if:
      - its INFO RS= contains any rsID listed in rsids_path, OR
      - its (CHROM, POS) is present in nonrsid_grch37_path rows.

    Writes VCF to vcf_out. Returns stats dict.
    """
    vcf_in = Path(vcf_in)
    vcf_out = Path(vcf_out)
    vcf_out.parent.mkdir(parents=True, exist_ok=True)

    rsids = load_rsids(rsids_path)
    positions = load_positions_from_nonrsid_rows(nonrsid_grch37_path)

    print(f"[load] rsids: {len(rsids):,} from {rsids_path}")
    print(f"[load] positions (GRCh37 non-rsid rows): {len(positions):,} from {nonrsid_grch37_path}")

    matched_rsids: Set[str] = set()
    matched_positions: Set[Tuple[str, int]] = set()

    kept = 0
    total = 0

    with vcf_in.open("r", encoding="utf-8", errors="replace") as fin, vcf_out.open("w", encoding="utf-8") as fout:
        for line in fin:
            if line.startswith("#"):
                fout.write(line)
                continue

            total += 1
            parts = line.rstrip("\n").split("\t")
            if len(parts) < 8:
                continue  # malformed
            chrom, pos_str, info = parts[0], parts[1], parts[7]

            # RSID match?
            rs_here = clinvar_info_rsids(info)
            rs_hit = bool(rs_here & rsids)

            # POS match?
            try:
                pos_int = int(pos_str)
            except ValueError:
                pos_int = -1
            pos_hit = (chrom, pos_int) in positions if pos_int >= 0 else False

            if rs_hit or pos_hit:
                fout.write(line)
                kept += 1
                if rs_hit:
                    matched_rsids |= (rs_here & rsids)
                if pos_hit:
                    matched_positions.add((chrom, pos_int))

    print(f"[vcf] processed: {total:,}  kept: {kept:,}")

    missing_rsids = len(rsids - matched_rsids)
    missing_positions = len(positions - matched_positions)

    print(f"[report] matched rsids: {len(matched_rsids):,} (missing {missing_rsids:,})")
    print(f"[report] matched positions: {len(matched_positions):,} (missing {missing_positions:,})")

    return {
        "vcf_in": str(vcf_in),
        "vcf_out": str(vcf_out),
        "rsids_loaded": len(rsids),
        "positions_loaded": len(positions),
        "records_processed": total,
        "records_kept": kept,
        "matched_rsids": len(matched_rsids),
        "matched_positions": len(matched_positions),
        "missing_rsids": missing_rsids,
        "missing_positions": missing_positions,
    }


In [ ]:
stats = filter_clinvar_with_rsids_and_positions(
    vcf_in="downloads/clinvar.vcf",
    rsids_path="work/23andme_rsids.txt",
    nonrsid_grch37_path="work/23andme_nonrsid_grch37.txt",
    vcf_out="work/clinvar.23andme.vcf",
)
stats


In [ ]:
# okay so Dawn says this is plausible and it could be that many 100ks of snps 
# that 23andme test for are non medical ancestry based ones with no clinical significance
# https://www.reddit.com/r/23andme/comments/3dd3lp/snp_coverage_analysiscomparisons_23andme_v3v4

In [ ]:
# lets save this as a .vcf.gz

In [ ]:
!head ./work/clinvar.23andme.vcf

In [ ]:
!mkdir -p ./results

In [ ]:
!bgzip -c ./work/clinvar.23andme.vcf > ./results/clinvar.23andme.vcf.gz
!tabix -p vcf ./results/clinvar.23andme.vcf.gz

In [ ]:
!bcftools view ./results/clinvar.23andme.vcf.gz > /dev/null

In [ ]:
import hashlib
from pathlib import Path

def write_md5(path: str):
    file = Path(path)
    md5 = hashlib.md5(file.read_bytes()).hexdigest()
    # Construct ClinVar-style line
    line = f"{md5}  {file.name}\n"
    # Write alongside the file with .md5 extension
    md5_path = file.with_suffix(file.suffix + ".md5")
    md5_path.write_text(line)
    print(f"✅ Wrote {md5_path}")
    print(line.strip())

# Example usage
write_md5("./results/clinvar.23andme.vcf.gz")